<a href="https://colab.research.google.com/github/maxzameer/Brainstorm/blob/main/deep%20learning/cat_vs_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats


 98% 1.05G/1.06G [00:04<00:00, 251MB/s]
100% 1.06G/1.06G [00:04<00:00, 272MB/s]


In [3]:
import zipfile

In [5]:
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [16]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization

In [8]:

train_df = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    validation_split = 0.2,
    subset = 'training',
    seed = 1337,
    batch_size = 32,    
    image_size = (256,256)
)

val_df = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    validation_split = 0.2,
    subset = 'validation',
    seed = 1337,
    image_size = (256,256)
)

Found 20000 files belonging to 2 classes.
Using 16000 files for training.
Found 20000 files belonging to 2 classes.
Using 4000 files for validation.


In [9]:
def normalize(image,label):
  image = tf.cast(image/255,tf.float32)
  return image,label

In [10]:
train_df = train_df.map(normalize)
val_df = val_df.map(normalize)

In [17]:

# create CNN model

model = Sequential()
model.add(Conv2D(32,kernel_size = (3,3),padding = 'valid',activation ='relu',input_shape =(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2 , padding = 'valid'))

model.add(Conv2D(32,kernel_size = (3,3),padding = 'valid',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2 , padding = 'valid'))

model.add(Conv2D(32,kernel_size = (3,3),padding = 'valid',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2 , padding = 'valid'))

model.add(Flatten())
model.add(Dense(128,activation ='relu'))
model.add(BatchNormalization())
model.add(Dense(64,activation ='relu'))
model.add(BatchNormalization())
model.add(Dense(1,activation ='sigmoid'))



In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 32)       0

In [18]:
model.compile(optimizer='adam',loss ='binary_crossentropy',metrics = ['accuracy'])


In [ ]:
history = model.fit(train_df,epochs=10,validation_data=val_df)


Epoch 1/10
500/500 [==============================] - 44s 86ms/step - loss: 0.6182 - accuracy: 0.6552 - val_loss: 1.9420 - val_accuracy: 0.5182
Epoch 2/10
500/500 [==============================] - 42s 83ms/step - loss: 0.5305 - accuracy: 0.7350 - val_loss: 0.6727 - val_accuracy: 0.6743
Epoch 3/10
500/500 [==============================] - 42s 84ms/step - loss: 0.4521 - accuracy: 0.7863 - val_loss: 0.5176 - val_accuracy: 0.7375
Epoch 4/10
500/500 [==============================] - 42s 83ms/step - loss: 0.3693 - accuracy: 0.8353 - val_loss: 0.6401 - val_accuracy: 0.7040
Epoch 5/10
365/500 [====================>.........] - ETA: 9s - loss: 0.2844 - accuracy: 0.8789